In [91]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import locale
import time
from datetime import datetime
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

from tqdm import tqdm

In [9]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

### object detail
id: int  
screen_name: @usuario  
tweet_count: int  
description: descripcion usuario 
location: ubicacion en desc  
url: url en desc  
listed_count: listas de las cuales forma parte  
created_at: cuenta creada el  
crawled_at: cuando se escrapeo  
favs_count: int  
followers_count: int  
following_count: int
verified: true/false

In [76]:
userDf = pd.DataFrame(columns=[
    'id',
    'screen_name',
    'complete_name',
    'tweet_count',
    'description',
    'location',
    'birthdate',
    'url',
    'listed_count',
    'created_at',
    'crawled_at',
    'favs_count',
    'followers_count',
    'following_count',
    'verified'
])

In [45]:
toScrap = pd.DataFrame(data=['ElGatoyLaCaja', 'matiNotFound', 'Floxie10', 'realdonaldtrump', 'merotegui'], columns=['screen_name'])
toScrap.head()

,screen_name
0,ElGatoyLaCaja
1,matiNotFound
2,Floxie10
3,realdonaldtrump
4,merotegui


In [92]:
def scrap_users(toScrap):
    for username in tqdm(toScrap):
        pageResponse = requests.get('https://twitter.com/'+username, headers=headersChrome, proxies=proxies)
        pageParsed = BeautifulSoup(pageResponse.text, 'html.parser')
        profileNavItems = {
            'tweets': 0,
            'following': 0,
            'followers': 0,
            'favorites': 0,
            'moments': 0,
            'lists': 0
        }
        for itemName in profileNavItems.keys():
            liSearch = pageParsed.findAll(attrs={'class':'ProfileNav-item--'+itemName})
            if(len(liSearch)>0):
                liValue = liSearch[0].findAll(attrs={'class':'ProfileNav-value'})[0]
                if( 'data-count' in  liValue.attrs.keys()):
                    profileNavItems[itemName] = liValue.attrs['data-count']
                else:
                    profileNavItems[itemName] = liValue.text
        completeName = pageParsed.findAll('h1', attrs={'class':'ProfileHeaderCard-name'})[0].findAll('a')[0].text
        completeName = completeName.replace(u'\xa0', u' ')
        completeName = completeName.replace(u'\n', ' ')
        bio = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-bio'})[0].text
        bio = bio.replace(u'\xa0', u' ')
        bio = bio.replace(u'\n', ' ')
        isVerified = False
        if(len(pageParsed.findAll(attrs={'class':'ProfileHeaderCard-name'})[0].findAll(attrs={'class':'Icon--verified'}))>0):
            isVerified = True
        userId = int(pageParsed.findAll(attrs={'class':'ProfileNav'})[0].attrs['data-user-id'])
        profileHeaderItems = {
            'location': False,
            'url': False,
            'joinDate': False,
            'birthdate': False
        }
        for itemName in profileHeaderItems.keys():
            liSearch = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-'+itemName})
            if(len(liSearch)>0):
                if('u-hidden' not in liSearch[0].attrs['class']):
                    liValue = liSearch[0].findAll(attrs={'class':'ProfileHeaderCard-'+itemName+'Text'})[0].text
                    liValue = re.sub(u'\n', ' ', liValue)
                    liValue = liValue.strip(' ')
                    liValue = liValue.replace(u'\xa0', u' ')
                    if(itemName == 'joinDate'):
                        liValue = re.sub(r'Se unió en ', '', liValue)
                        liValue = re.sub(r' de ', ' ', liValue)
                        liValue = datetime.strptime(liValue, '%B %Y')
                        liValue = pd.Timestamp(ts_input=liValue)
                    if(itemName == 'birthdate'):
                        liValue = re.sub(r'Fecha de nacimiento ', '', liValue)
                    profileHeaderItems[itemName] = liValue
        userData = {
            'id': userId,
            'screen_name': username,
            'complete_name': completeName,
            'description': bio,
            'verified': isVerified,
            'crawled_at': pd.Timestamp.now(),
            'location': profileHeaderItems['location'],
            'url': profileHeaderItems['url'],
            'created_at': profileHeaderItems['joinDate'],
            'birthdate': profileHeaderItems['birthdate'],
            'tweet_count': profileNavItems['tweets'],
            'listed_count': profileNavItems['lists'],
            'favs_count': profileNavItems['favorites'],
            'followers_count': profileNavItems['followers'],
            'following_count': profileNavItems['following']
            
        }
        userDf.loc[userData['id']] = userData

scrap_users(toScrap.screen_name.values)

userDf.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.72s/it]


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
2180922097,2180922097,ElGatoyLaCaja,El Gato y La Caja,15140,Ciencia en lugares.,False,20 de febrero,elgatoylacaja.com.ar/bancar/,0,2013-11-01 00:00:00,2019-06-28 16:36:15.873309,24490,58801,646,True
1090221700913029123,1090221700913029123,matiNotFound,Matias,0,A la grande le puse cuca,False,False,matiasgf.com,0,2019-01-01 00:00:00,2019-06-28 16:36:17.169368,26,7,15,False
118441514,118441514,Floxie10,El mundo de Floxie 🌎,104449,Travel Blogger. Bicho de aeropuerto. ahora en ...,"Buenos Aires, Argentina",10 de octubre,floxie.com.ar,5,2010-02-01 00:00:00,2019-06-28 16:36:19.134223,111440,47024,902,False
25073877,25073877,realdonaldtrump,Donald J. Trump,42588,45th President of the United States of America,"Washington, DC",False,Instagram.com/realDonaldTrump,0,2009-03-01 00:00:00,2019-06-28 16:36:20.878330,7,61480170,47,True
565841194,565841194,merotegui,María Mercedes,14641,no soy tan simple como para no advertir que no...,False,False,False,0,2012-04-01 00:00:00,2019-06-28 16:36:22.599968,21636,525,2960,False
